# Pós Ciência dos Dados - Aplicações de Aprendizado de Máquina

## Atividade 1

Utilize as células a seguir como base para carregar as informações, e a partir da última célula desse modelo, desenvolva seu raciocínio e como seria a abordagem do seu grupo para esse problema. *Não altere as células apresentadas aqui para não danificar a leitura dos dados.*

Os dados estão mascarados, mas basicamente, a variável *y* aponta para quando uma determinada ação vai subir ou vai descer na bolsa.

---

**Nome dos integrantes**

Aluno 1:

Aluno 2:

Aluno 3:

Aluno 4:

---

Lembrem-se, o grupo deve ter no máximo 4 integrantes por grupo, e deverão ser entregues até **08/02 às 23h59** esse notebook preenchido com os experimentos realizados e as conclusões observadas, além de um PDF com 5 slides para o grupo apresentar em 3 minutos para o restante da sala.

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model

# deixa fixo o fator de aleatoriedade
np.random.seed(0)

In [2]:
# carrega os dados
data = pd.read_csv('training_data.csv', header=0)

data = data.drop([
    'id', 'era', 'data_type', 'target_charles', 'target_elizabeth',
    'target_jordan', 'target_ken', 'target_frank', 'target_hillary'],axis=1)

In [4]:
# transforma o CSV em numpy
features = [f for f in list(data) if "feature" in f]
X = data[features]
y = data['target_bernie']

In [5]:
# exibe quantidade de amostras e atributos
XLista = df_split = np.array_split(X, 100)
X0 = XLista[0]


YLista = df_split = np.array_split(y, 100)
y0 = YLista[0]

In [6]:
X0.shape

(5028, 50)

In [ ]:
# a partir daqui é com você...
# 1) separe 30% dos dados para teste e utilize os outros 70% como achar melhor
# 2) lembre-se de aplicar os conceitos vistos em aula

In [7]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca = PCA(n_components=2)
X_r = pca.fit_transform(X0)
ids=['0','1']
colors = ['blue', 'red']
for color, i, tgname in zip(colors, [0,1],ids):
    plt.scatter(X_r[y0 == i, 0], X_r[y0 == i, 1], color=color, alpha=0.2, label=tgname)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of Bolsa dataset')

Text(0.5,1,'PCA of Bolsa dataset')

In [15]:
#knn simples
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
X_train2, X_test2, y_train2, y_test2 = train_test_split(X0, y0, test_size=0.3, random_state=42)

knn2 = KNeighborsClassifier(n_neighbors=3)

In [16]:
knn2.fit(X_train2, y_train2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [17]:
y_pred2 = knn2.predict(X_test2)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test2, y_pred2)

0.4886295829742094

In [23]:
# utilizando validação cruzada com cross_val_score
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])
scores = cross_val_score(pipeline, X0, y0, cv=5) # 5 execuções diferentes com 20% dos dados para teste

print('Accuracy - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

Accuracy - 49.80 +- 0.11


In [ ]:
# separa-se uma parcela para encontrar os melhores parâmetros (5% do original)
data_gs, data_cv, target_gs, target_cv = train_test_split(X0, y0, test_size=0.7, random_state=42, stratify=y0)

# uma forma automática de StandardScaler + KNN
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])

# utiliza-se GridSearchCV para achar os melhores parâmetros
from sklearn.model_selection import GridSearchCV
parameters = {'clf__n_neighbors': [1,2,3,4,5], 'clf__weights' : ['uniform','distance']} # quais parâmetros e quais valores serão testados
clf = GridSearchCV(pipeline, parameters, cv=3, iid=False) # clf vai armazenar qual foi a melhor configuração
clf.fit(data_gs, target_gs)

# utilizando validação cruzada para avaliar o modelo
scores = cross_val_score(clf, data_cv, target_cv, cv=5)

print('Accuracy - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))